In [ ]:
!pip install PyPDF2 -q
!pip install transformers -q
!pip install streamlit -q
!pip install SentencePiece -q
!pip install PyMuPDF -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 20.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from PyPDF2 import PdfReader
import pymupdf

PDFREADER vs PYMUPDF

In [ ]:
file_name = '/content/Operational_range_bounding_of_spectroscopy_models_with_anomaly_detection_draft.pdf'

In [ ]:
from PyPDF2 import PdfReader

reader = PdfReader('/content/Operational_range_bounding_of_spectroscopy_models_with_anomaly_detection_draft.pdf')
num_pages = len(reader.pages)
for x in range(num_pages):
  page_wise_text = []
  page = reader.pages[x]
  text = page.extract_text()

In [ ]:
type(text)

str

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import BartForConditionalGeneration, BartTokenizer
import textwrap

model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True, padding='longest') # Added Padding
summary_ids = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
formatted_summary = "\n".join(textwrap.wrap(summary, width=80))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
formatted_summary

'Research funded by the ESA Science Faculty Research Projects "Machine Learning\nQuality Assurance in Ariel and beyond" and "Ariel Machine Learning Data" The\nstudy was published in the journal The Astronomical Journal 159, 159 (2020). It\nis an updated version of an earlier version of this article which was published\nearlier this year.'

In [ ]:
def pdf_to_dataframe(input_file):
  metadata = []
  pdf = pymupdf.open(input_file)
  # Get page content as dict and blocks of data
  for page in pdf:
    pages = page.get_text('dict')
    blocks = pages['blocks']
    # Extract metadata from blocks and filter for text, font size and font name
    for block in blocks:
      if 'lines' in block.keys():
        spans = block['lines']
        for span in spans:
          data = span['spans']
          for x in data:
            metadata.append((x['text'], x['size'], x['font']))
  pdf.close()
  df = pd.DataFrame(metadata,columns = ['text','size','font'])
  return df

In [ ]:
df = pdf_to_dataframe(file_name)
print(df['font'].value_counts())

font
Kp-Regular              468
DejaVuSans              156
Kp-Medium                63
Kp-Italic                55
Kp-Expert-Regular        26
Kp--M-Italic             22
DejaVuSans-Bold          12
DejaVuSans-Oblique        7
Kp--M-Sy-Regular          5
Tt-Kp-Regular             4
Kp-Companion-Regular      2
Cmsy10                    2
Kp-Expert-Medium          1
Kp--M-Ex-Regular          1
MSBM10                    1
Kp-SmallCaps-Regular      1
Name: count, dtype: int64


In [ ]:
# Kp-Regular holds actual text so filter for that
df = df.loc[df['font'] == 'Kp-Regular']
# Column to list for preprocessing
# corpus = ' '.join(df['text'].tolist()) # original
corpus = ' '.join(df['text'])
corpus

'DRAFT version. Last modified: 2024-05-30 22:03:17Z Operational range bounding of spectroscopy models with anomaly detection Luís F. Simões 1 , Pierluigi Casale 1 , Marília Felismino 1 , Kai Hou Yip 2 , Ingo P. Waldmann 2 , Giovanna Tinetti 2 , Theresa Lueftinger 3 1 2 3 As Machine Learning adoption grows across risk- averse industries such as aerospace, various e orts are underway to design the validation processes that will ensure safety in complex operational settings [1–5].  being advocated by such e orts consists of a safety mechanism that continu- ously monitors system observables, and intervenes if it judges the model will exceed the bounds of a safe domain for a particular input. Unlike monitor- ing systems that run asynchronously from the pre- diction model and perform evaluations over assem- bled production data [6], the safety cage architecture runs a monitoring system in parallel with the predic- tion model, and operates at an individual sample level. This places explicit b

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import BartForConditionalGeneration, BartTokenizer

In [ ]:
import textwrap
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

inputs = tokenizer.encode("summarize: " + corpus, return_tensors="pt", max_length=1024, truncation=True, padding='longest') # Added Padding
summary_ids = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
formatted_summary = "\n".join(textwrap.wrap(summary, width=80))

In [ ]:
formatted_summary

'Machine Learning is being used to improve safety in complex operational\nsettings. Atmospheres of exoplanets are studied using transit spectroscopy. The\nAriel Mission is a European Space Agency (ESA) M4 mission that will conduct the\nfirst compre- hensive study of 1000 exoplanet in our galactic neigh- bourhood.\nWe conduct our experiments over the datasets from the 2019 and 2021 Ariel Data\nChal- lenges.'

In [ ]:
model_name = "facebook/bart-large"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name,forced_bos_token_id=0)

inputs = tokenizer.encode("summarize: " + corpus, return_tensors="pt", max_length=1024, truncation=True, padding='longest') # Added Padding
summary_ids = model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
formatted_summary = "\n".join(textwrap.wrap(summary, width=80))

In [ ]:
formatted_summary

'summarize: DRAFT version. Last modified: 2024-05-30 22:03:17Z Operational range\nbounding of spectroscopy models with anomaly detection Luís F. Simões 1,\nPierluigi Casale 1, Marília Felismino 1, Kai Hou Yip 2, Ingo P. Waldmann 2,\nGiovanna Tinetti 2, Theresa Lueftinger 3 1 2 3 As Machine Learning adoption\ngrows across risk- averse industries such as aerospace, the development of\nautomated systems that monitor the generation the model’s oper- tional range,\nand intervene if it judges the model will exceed the bounds of a safe domain for\na particular input. Ensuring the'

LOOK AT PEGASUS

In [ ]:
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')

batch = tokenizer(corpus, truncation=True, padding='longest', return_tensors="pt")
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tgt_text

['Operational range bounding of spectroscopy models with anomaly detection Lus F. Simes 1, Pierluigi Casale 1, Marlia Felismino 1, Kai Hou Yip 2, Ingo P.']

In [ ]:
!npm install -g localtunnel

In [ ]:
%%writefile app.py
import streamlit as st
import os
from PyPDF2 import PdfReader
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import BartForConditionalGeneration, BartTokenizer
from io import StringIO

# Intro Text
st.markdown('# PDF summarization using Pegasus')
st.header('''PDF size must be below 200MB. Only 1 PDF per time, time will vary depending on size''')

input_file = st.file_uploader('Upload a PDF file')

# Save uploaded file
if input_file:
  with open(os.path.join(input_file.name),'wb') as f:
        f.write(input_file.getbuffer())
  st.success('Saved File')

st.markdown('# Summary')

# Load tokeniser
# tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
# Load model
# model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large", forced_bos_token_id=0)

if input_file:
    # Load the PDF reader object
    data = open(input_file.name, 'rb')
    reader = PdfReader(data)
    num_pages = len(reader.pages)

    # Choose which pages to summarise (all is default)
    values = st.slider(
     'Summarize page number between:',
     min_value= 1, max_value= num_pages,
     value= (1,num_pages),
     step = 1)

    with st.spinner('Preparing summary. Please wait...'):
      for x in range(values[0]-1,values[1]-1):

        # Extract text from each page
        page_wise_text = []
        page = reader.pages[x]
        text = page.extract_text()

        # Tokenise text
        tokens = tokenizer(text, truncation=True, padding='longest', return_tensors='pt')

        # Summarise text
        summary = model.generate(**tokens)

        # Decode tokens and add to list
        decoded_output = tokenizer.decode(summary[0])
        page_wise_text.append(decoded_output)

        # Show Summary
        st.markdown('Page number: {}'.format(x+1))
        st.text_area(label ="",value=page_wise_text, placeholder="Please upload a PDF to get it's summary", height = 100, key=x+1)
        st.markdown('')

In [ ]:
!curl ipv4.icanhazip.com

In [ ]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501